## Initialization
#### Opens data and defines basic parameters

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Opens data
file              = "Data_Scheduling_AN.xlsx"
rootpath          = "./"
sheets            = pd.ExcelFile(os.path.join(rootpath, file))
availability      = pd.read_excel(sheets,'Availability AN',header = 2)
tutor_class       = pd.read_excel(sheets,'Tutor_Class AN')
rooms             = pd.read_excel(sheets,'Rooms AN')
slots             = pd.read_excel(sheets,'Slots AN')
days              = pd.read_excel(sheets,'Days AN')

# Defines quantity of each variable
num_tutors        = len(tutor_class["Tutor ID"].unique())
num_slots         = len(slots)
num_rooms         = len(rooms)
num_days          = len(days)

# Defines what variable corresponds to what index
ind_tutors        = 0
ind_slots         = 1
ind_days          = 2
ind_rooms         = 3

# Creates decission variable
x = np.zeros((num_tutors, num_slots, num_days, num_rooms))

# Rearranges availability have same index format as decission variable
mon_availability   = np.array(availability[['S01-Mon','S02-Mon','S03-Mon','S04-Mon','S05-Mon']])
tue_availability   = np.array(availability[['S01-Tue','S02-Tue','S03-Tue','S04-Tue','S05-Tue']])
wed_availability   = np.array(availability[['S01-Wed','S02-Wed','S03-Wed','S04-Wed','S05-Wed']])
tutor_availability = np.array([mon_availability,tue_availability,wed_availability])
tutor_availability = np.moveaxis(tutor_availability,0,-1)

print(f'Shape of decission variable x: {np.shape(x)}')
print(f'Shape of tutor availability x: {np.shape(tutor_availability)}')

# Number of times a tutor needs to be scheduled
allocation_num  = np.array(tutor_class['Tutor ID'].value_counts()[availability['Slot']])

Shape of decission variable x: (21, 5, 3, 6)
Shape of tutor availability x: (21, 5, 3)


## Generation of feasible schedules
#### Randomizes allocation

In [2]:
def generate_random_index(allocation_number,past_indeces):
    # Generates an index that hasn't already been considered
    boolean = False
    
    while not(boolean):
        
        # Generates random index
        random_index = np.random.randint(0, allocation_number)
        
        # If index has not been considered, that index is returned
        # Otherwise repeat process until a feasible index is found
        if random_index not in past_indeces:
            boolean = True
   
    return random_index

In [3]:
def generate_random_room(rooms):
    # Gets an index of a room that is not occupied
    boolean = False
    
    # If there are no rooms available, returns -1
    if sum(rooms) == num_rooms:
        return 'No Rooms Available'
    
    # Looks up rooms which are available
    available_rooms = np.where(rooms == 0)
    
    while not(boolean):
        
        # Generates random index
        random_room = np.random.randint(0, num_rooms)
        
        # If index corresponds to available room, index is returned
        # Otherwise repeat process until a feasible index is found
        if random_room in available_rooms[0]:
            boolean = True
        
    return random_room

In [4]:
def generate_feasible_schedule():
    
    feasible = False
    attempts = 1
    
    while not(feasible):
        
        problem = False
        
        x = np.zeros((num_tutors, num_slots, num_days, num_rooms))
        
        for tutor in range(num_tutors):
            #print(f'Tutor: {tutor}')

            # Gets all slots and days tutor is available
            possible_allocations = np.where(tutor_availability[tutor] == 1)

            # For each tutor, saves indeces of allocated spaces such that tutor is not scheduled to the same time
            past_indeces = []

            #print(f'Possible Allocations: {len(possible_allocations[0])}')
            #print(f'Number of allocations needed: {allocation_num[tutor]}')

            # If allocation number greater than possible allocations: problem is infeasable
            if len(possible_allocations[0]) < allocation_num[tutor]:
                print('Problem is infeasable as number of allocations needed is greater than preference number')
                print('Please review data')
                break

            # Allocates tutor to a room
            for i in range(allocation_num[tutor]):

                # Random index (doesn't double schedule the tutor)
                random_index = generate_random_index(len(possible_allocations[0]),past_indeces)

                # Gets the slot and day of randomized index
                random_slot = possible_allocations[0][random_index]
                random_day  = possible_allocations[1][random_index]

                # Randomizes room for allocation, and selects a room that is available 
                available_rooms = np.apply_over_axes(np.sum, x, ind_tutors)
                available_rooms = available_rooms[0,random_slot,random_day]

                # Gets room that is empty
                random_room = generate_random_room(available_rooms)

                # If can't allocate room: break and restart the porcess
                if random_room == 'No Rooms Available':
                    problem = True
                    #print('No Rooms Available')
                    break

                # Fix so it escapes loop and add while
                x[tutor,random_slot,random_day,random_room] = 1

                past_indeces.append(random_index)
                #print("")
        
        if problem:
            problem  = False
            attempts += 1
        else:
            feasible = True
                
            
    return (attempts,x)


## Printing a feasible schedule
#### Given the decission variable $x_{tsdr}$, it produces a schedule

In [5]:
def print_schedule(instance):
    # Prints out schedule

    # Creates empty schedule
    columns  = pd.MultiIndex(levels = [days['Day'].values,slots['Slot ID'].values],codes = [[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2],[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4]])
    indeces  = rooms['Room ID'].values
    schedule = pd.DataFrame(np.zeros((num_rooms,num_days*num_slots)),columns = columns,index = indeces)
    schedule = schedule.astype(str)

    # Populates empty schedule with tutors dictated by the decission variable
    for day in range(num_days):
        for room in range(num_rooms):
            for slot in range(num_slots):

                # Finds who is the tutor in that room at that slot at that day 
                tutor_index = np.where(instance[:,slot,day,room] == 1)

                # Converts from index form
                day_name  = days['Day'][day]
                slot_name = slots['Slot ID'][slot]
                room_name = rooms['Room ID'][room]

                # Populates cell with tutor name, and if no tutor allocated, leaves it empty    
                if tutor_index[0].size != 0:
                    # Gets tutor Id
                    tutor_id = availability['Slot'][tutor_index[0]]

                    # Updates schdule
                    schedule.loc[room_name,day_name][slot_name] = tutor_id.values[0]

                else:
                    schedule.loc[room_name,day_name][slot_name] = ''  
    return schedule

## Genrating a random feasible schedule

In [6]:
attempts, x = generate_feasible_schedule()
print(f'Number of attempts needed to create feasible schedule: {attempts}')
schedule = print_schedule(x)
schedule

Number of attempts needed to create feasible schedule: 1


Monday                  Tuesday                     Wednesday            \
       S01  S02 S03 S04 S05     S01  S02  S03  S04  S05       S01  S02  S03   
R01         T17                 T16            T05  T16                       
R02         T15                      T07  T04  T06  T04                       
R03    T09                      T17  T21  T05  T10                      T12   
R04                             T18       T08  T02                            
R05    T15                      T20  T13  T21                           T11   
R06                             T04       T06  T14  T02            T10  T01   

               
     S04  S05  
R01  T03  T19  
R02            
R03  T13       
R04            
R05  T12       
R06  T19

# Objective function

## 1) The number of days a tutor has to come into work should be minimized

In [7]:
def total_days_visted_by_tutors(instance):
    
    # Sums over rooms and slots
    sum_over_rooms_and_slots = np.squeeze(np.apply_over_axes(np.sum, instance, [ind_slots,ind_rooms]))

    count = 0
    
    # Iterates over each tutor
    for j in range(num_tutors):
        
        # Counts the days a tutor has to teach
        count += np.count_nonzero(sum_over_rooms_and_slots[j])
    
    return count

## 2) The number of rooms used each day should be minimized

In [8]:
def total_rooms_used_in_week(instance):
    
    # Sums over slots and tutors
    sum_over_tutors_and_slots = np.squeeze(np.apply_over_axes(np.sum, instance, [ind_tutors,ind_slots]))

    count = 0

    # Iterates over each day
    for j in range(num_days):
        
        # Counts the rooms that are used each day
        count += np.count_nonzero(sum_over_tutors_and_slots[j])
    
    return(count)

## 3) The amount of time a tutor waits in between tutorials should me minimized

In [9]:
def tutors_in_idle(instance):
    
    # Sums over every room
    sum_over_rooms = np.squeeze(np.apply_over_axes(np.sum, instance, ind_rooms))

    idle_time = 0
    
    # Iterates over each tutor and each day
    for t in range(num_tutors):
        for d in range(num_days):

            # Gets each tutor's schedule for a given day
            day_schedule = sum_over_rooms[t,:,d].astype(int)

            # Looks at the slots which have been assign to
            indeces = np.where(day_schedule == 1)[0]

            # Calculates the time tutor is waiting around in between tutorials
            if len(indeces) > 1:
                
                idle_time += max(indeces) - min(indeces) + 1 - len(indeces)
                
    return idle_time

## 4) The amount of times tutors should move rooms in between tutorials should be minimized

In [10]:
def times_tutors_move_rooms_with_b2b_tutorials(instance):
    
    times_moved = 0

    # Iterates over each tutor over each day
    for t in range(num_tutors):
            for d in range(num_days):


                # Iterates over each slot
                for s in range(num_slots-1):


                    # If that tutor is allocated to that slot in that day
                    if sum(instance[t,s,d,:] == 1):

                        # Gets what room it is allocated to
                        room_index = np.where(instance[t,s,d,:] == 1)[0]

                        # Check if tutor is allocated on the next slot
                        if sum(instance[t,s+1,d,:] == 1):

                            # Gets what room it is allocated to next slot
                            next_room_index = np.where(instance[t,s+1,d,:] == 1)[0]

                            # Penalizes if rooms are not the same
                            if room_index != next_room_index:
                                times_moved += 1

    return times_moved

### Combining all objectives

In [11]:
def calculate_fitness(individual):
    
    number_of_visits_cost = total_days_visted_by_tutors(individual)
    rooms_used_cost       = total_rooms_used_in_week(individual)
    tutors_waiting_cost   = tutors_in_idle(individual)
    moving_rooms_cost     = times_tutors_move_rooms_with_b2b_tutorials(individual)
    
    #print(f'Total number of visits = {number_of_visits_cost}')
    #print(f'Total number of rooms used over the week = {rooms_used_cost}')
    #print(f'Total time tutors spend waiting = {tutors_waiting_cost}')
    #print(f'Total amount of times tutors having to moves rooms = {moving_rooms_cost}')
    
    return number_of_visits_cost + rooms_used_cost + tutors_waiting_cost + moving_rooms_cost

In [12]:
schedule = print_schedule(x)
schedule

Monday                  Tuesday                     Wednesday            \
       S01  S02 S03 S04 S05     S01  S02  S03  S04  S05       S01  S02  S03   
R01         T17                 T16            T05  T16                       
R02         T15                      T07  T04  T06  T04                       
R03    T09                      T17  T21  T05  T10                      T12   
R04                             T18       T08  T02                            
R05    T15                      T20  T13  T21                           T11   
R06                             T04       T06  T14  T02            T10  T01   

               
     S04  S05  
R01  T03  T19  
R02            
R03  T13       
R04            
R05  T12       
R06  T19

In [13]:
calculate_fitness(x)

50

# Genetic Algorithm

In [14]:
def get_fitness_scores(population):
    
    fitness_scores = np.zeros(POPULATION_NUMBER)

    for i in range(len(population)):
        
        fitness_score = -1*calculate_fitness(population[i])
        
        fitness_scores[i] = fitness_score
        
    return fitness_scores
        

In [15]:
# Initializes population
POPULATION_NUMBER = 200
population = []

for i in range(POPULATION_NUMBER):
    _ , x = generate_feasible_schedule()
    population.append(x)
    
# Iterates over generations
MAX_GENERATIONS    = 100

for generation in range(MAX_GENERATIONS):
    
    next_genration = np.zeros(POPULATION_NUMBER)
    
    # Calculates the fitness score for each individual in the population
    fitness_scores = get_fitness_scores(population)
    
    # Gets indeces of two highest scoring individuals
    top_indeces      = np.flip(np.argpartition(fitness_scores, -2[-2:]))
    top_index        = top_indeces[0]
    second_top_index = top_indeces[1]
    
    # By elitism, individuals are selected to go onto the next generation
    next_generation[0] = population[top_index]
    next_generation[1] = population[second_top_index]
    
    # Relax condition of where tutors are allocated
    
    
    
    
                                  
                                  
    
    
    
    

    

<>:20: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:20: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
/var/folders/5s/lrmbcsvs05s1y30_zq4mzvwm0000gn/T/ipykernel_37268/3126777301.py:20: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  top_indeces      = np.flip(np.argpartition(fitness_scores, -2[-2:]))
/var/folders/5s/lrmbcsvs05s1y30_zq4mzvwm0000gn/T/ipykernel_37268/3126777301.py:20: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  top_indeces      = np.flip(np.argpartition(fitness_scores, -2[-2:]))
/var/folders/5s/lrmbcsvs05s1y30_zq4mzvwm0000gn/T/ipykernel_37268/3126777301.py:20: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  top_indeces      = np.flip(np.argpartition(fitness_scores, -2[-2:]))


TypeError: 'int' object is not subscriptable

In [92]:
child = np.zeros((num_tutors, num_slots, num_days))

parent1 = np.squeeze(np.apply_over_axes(np.sum, population[1], ind_rooms))
parent2 = np.squeeze(np.apply_over_axes(np.sum, population[2], ind_rooms))

for t in range(num_tutors):
    
    # Randomly chooses if to take genes from parent one or two
    num = np.random.randint(2)
    
    if num == 1:
        child[t] = parent1[t]
    else:
        child[t] = parent2[t]
        
        
extended_child = np.zeros((num_tutors, num_slots, num_days, num_rooms))

count = 0
room_indeces = np.zeros([num_slots,num_days])

for t in range(num_tutors):
    
    for s in range(num_slots):
        for d in range(num_days):
            
            if child[t,s,d] == 1:
                r = int(room_indeces[s,d])
                extended_child[t,s,d,r] = 1
                
                room_indeces[s,d] = r + 1
                
for d in range(num_days):
    for s in range(num_slots):
        for r in range(num_rooms):
            
            # Is there a tutor allocated to this slot on this day in this room?
            if sum(extended_child[:,s,d,r]) > 0:
                
                tutor_index = np.squeeze(np.where(extended_child[:,s,d,r] == 1))
                
                # Is this tutor back to back?
                if s < 4:
                    
                    # Is there a tutor allocated to next room
                    if sum(extended_child[tutor_index,s+1,d,:]) > 0:
                        
                        # Finds where the tutor is allocated to
                        next_room_index = np.squeeze(np.where(extended_child[tutor_index,s+1,d,:] == 1))
                        
                        # If tutor has b2b tutorials but they are in different rooms
                        if r!= next_room_index:
                        
                            
                            #extended_child[tutor_index,s+1,d,r]               = 1
                            #extended_child[tutor_index,s+1,d,next_room_index] = 0

                            
                            # Checks if there was a tutor in that room
                            if sum(extended_child[:,s+1,d,next_room_index]) > 0:
                                
                                next_tutor_index = np.squeeze(np.where(extended_child[:,s+1,d,r] == 1))
                             #   
                                print(availability['Slot'][tutor_index],rooms['Room ID'][r],rooms['Room ID'][next_room_index])
                                print(availability['Slot'][next_tutor_index])
                                
                                #extended_child[next_tutor_index,s+1,d,next_room_index] = 0
                                #extended_child[next_tutor_index,s+1,d,r] = 1
                                
                    



            
                
                        

schedule = print_schedule(extended_child)
schedule   

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [36]:
r

0.0

In [21]:
np.sum(child,0)

array([[2., 3., 1.],
       [2., 5., 2.],
       [0., 5., 3.],
       [0., 5., 2.],
       [0., 3., 1.]])

In [28]:
np.shape(np.arange(num_tutors))

(21,)

In [18]:
sum(sum(sum(sum(extended_child))))

34.0

In [32]:
dictionary

{0: 1,
 1: 1,
 2: 1,
 3: 2,
 4: 2,
 5: 2,
 6: 1,
 7: 1,
 8: 1,
 9: 2,
 10: 1,
 11: 2,
 12: 2,
 13: 2,
 14: 2,
 15: 2,
 16: 1,
 17: 2,
 18: 3,
 19: 2,
 20: 1}

In [ ]:
top_indeces      = np.flip(np.argpartition(fitness_scores, -2)[-2:])
top_index        = top_indeces[0]
second_top_index = top_indeces[1]

In [ ]:
schedule = print_schedule(population[top_index])
schedule

In [ ]:
schedule = print_schedule(population[2])
schedule

In [ ]:
def switching(x):

    x_copy = copy.deepcopy(x)
    change = False

    if sum(daily_schedule) > 1:
        back_to_back       = False
        index_back_to_back = 0

        # Does this tutor have back to back tutorials
        if s_random < 1:
            if sum(x[t_random,s_random+1,d_random,:]) == 1:
                back_to_back       = True
                index_back_to_back = s_random+1


        elif s_random > 0 and s_random < 4:
            if sum(x[t_random,s_random+1,d_random,:]) == 1:
                back_to_back       = True
                index_back_to_back = s_random+1
            elif sum(x[t_random,s_random-1,d_random,:]) == 1:
                back_to_back       = True
                index_back_to_back = s_random-1



        elif s_random > 3:
            if sum(x[t_random,s_random-1,d_random,:]) == 1:
                back_to_back       = True
                index_back_to_back = s_random-1

        # Are these back to back tutorails in different classrooms
        next_room_index = np.where(x[t_random,index_back_to_back,d_random,:] == 1)[0]


        if back_to_back and r_random != next_room_index:


            # Checks if there was a tutor in that particular classroom
            other_tutor_index = np.where(x[:,s_random,d_random,next_room_index])

            print(other_tutor_index)

            if len(other_tutor_index) > 0:
                x_copy[other_tutor_index,s_random,d_random,next_room_index] = 0
                x_copy[other_tutor_index,s_random,d_random,r_random] = 1

            # Swaps room
            x_copy[t_random,s_random,d_random,r_random]        = 0
            x_copy[t_random,s_random,d_random,next_room_index] = 1
            
            change = True
            
        








    # are they back to back#
    #Yes: ranomdly switch to desired rooms

    # No:  -randomly switch slots


    # Else: switch them to another aailable time hopefully on an other day

    # 
    return change, x_copy


In [ ]:
schedule = print_schedule(population[2])
schedule

In [ ]:
# Picks a random spot that is populated
x_copy = copy.deepcopy(population[2])

import copy
for random in range(34):

    t_random = np.where(x_copy == 1)[0][random]
    s_random = np.where(x_copy == 1)[1][random]
    d_random = np.where(x_copy == 1)[2][random]
    r_random = np.where(x_copy == 1)[3][random]

    #print(t_random,s_random,d_random,r_random)

    # Does this person have more than one tute this day
    daily_schedule = np.sum(x_copy[t_random,:,d_random,:],1).astype(int)

    day_name  = days['Day'][d_random]
    slot_name = slots['Slot ID'][s_random]
    room_name = rooms['Room ID'][r_random]
    tutor_id = availability['Slot'][t_random]

    change, x_copy = switching(x_copy)
    if change:
        print(tutor_id,slot_name,day_name,room_name)
        print(daily_schedule)

schedule = print_schedule(x_copy)
schedule